In [1]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
from flask import Flask, request, jsonify
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import colorsys
from skimage.color import rgb2lab

app = Flask(__name__)

warm_tone = [255, 255, 0]  # 노란색
cool_tone = [0, 0, 255]  # 파란색

def zero_shot_classification(color_vector):
    warm_similarity = np.dot(color_vector, warm_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(warm_tone))
    cool_similarity = np.dot(color_vector, cool_tone) / (np.linalg.norm(color_vector) * np.linalg.norm(cool_tone))

    if warm_similarity > cool_similarity:
        return "웜톤"
    else:
        return "쿨톤"

def classify_image(image):
    image = np.array(image)
    image = image / 255.0  # 이미지를 0~1 범위로 정규화
    height, width, _ = image.shape
    colors = np.reshape(image, (height * width, 3))  # 이미지의 모든 픽셀을 1차원 벡터로 변환

    color_labels = []
    for color in colors:
        color_label = zero_shot_classification(color)
        color_labels.append(color_label)

    warm_tone_count = color_labels.count('웜톤')
    cool_tone_count = color_labels.count('쿨톤')

    warm_tone_ratio = warm_tone_count / len(color_labels) * 100
    cool_tone_ratio = cool_tone_count / len(color_labels) * 100

    return f"웜톤: {warm_tone_ratio:.2f}%\n쿨톤: {cool_tone_ratio:.2f}%"

@app.route('/process_personal_color', methods=['POST'])
def process_personal_color():
    try:
        file = request.files['image']
        img = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
        result = classify_image(img)

        return jsonify({"result": result})
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
